In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import tensorflow as tf
import numpy as np

In [2]:
img_size=50
x_train = np.load("X_train_" + str(img_size)+".npy")
x_test = np.load("X_test_" + str(img_size)+".npy")
y_train = np.load("y_train_" + str(img_size)+".npy")
y_test = np.load("y_test_" + str(img_size)+".npy")

In [3]:
x = tf.placeholder(tf.float32, [None, img_size,img_size,3],name="x")
x = tf.reshape(x,[-1,img_size,img_size,3])

W_conv1=tf.Variable(tf.truncated_normal([5, 5,3,32],stddev=0.1),name="W_conv1")

b_conv1=tf.Variable(tf.truncated_normal([1,1,1,32],stddev=0.1),name="b_conv1")

z_conv_1=tf.nn.conv2d(x, W_conv1, strides=[1, 1, 1, 1], padding='SAME')+b_conv1

h_conv_1 = tf.nn.relu(z_conv_1)

h_pool_1=tf.nn.max_pool(h_conv_1, ksize=[1, 2, 2, 1],
                                strides=[1, 2, 2, 1], padding='SAME')
size = img_size / 2

fc_0=tf.reshape(h_pool_1,[-1,size*size*32])

W_fc1 = tf.Variable(tf.truncated_normal([size*size*32,1024],stddev=0.1),name="W_fc1")

b_fc1 = tf.Variable(tf.truncated_normal([1,1024],stddev=0.1),name="b_fc1")

h_fc1 = tf.nn.relu(tf.matmul(fc_0, W_fc1) + b_fc1)

W_fc2 = tf.Variable(tf.truncated_normal([1024, 120],stddev=0.1),name="W_fc2")

b_fc2 = tf.Variable(tf.truncated_normal([1,120],stddev=0.1),name="b_fc2")

z_pred=tf.matmul(h_fc1, W_fc2) + b_fc2

y_pred = tf.nn.softmax(z_pred)

y = tf.placeholder(tf.float32, [None, 120],name="y")

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                                    _sentinel=None,
                                    labels=y,
                                    logits=z_pred,
                                    dim=-1,
                                    name=None
                                )
                              )

train_step = tf.train.AdamOptimizer(1e-2).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


epochs = 2

epoch_run=tf.Variable(0,trainable=False, name="epochs_run")

batch_size=150

saver = tf.train.Saver()

with tf.Session() as sess:

    writer = tf.summary.FileWriter('/home/aman/dogs/OpenCV/graphs', sess.graph)

#     sess.run(tf.global_variables_initializer())

    saver.restore(sess, "/home/aman/dogs/OpenCV/models/"+str(img_size)+"model.ckpt")

    print("Model restored.")

    for i in range(epochs):

        batch_count=18000/batch_size

        for batch in range(batch_count):

            print("epoch {} and mini-batch {} : Cross-entropy error is {} and training accuracy is {}".
                  format(sess.run(epoch_run)+i+1,
                         batch+1,
                         sess.run(cross_entropy,
                                  feed_dict={x:x_train[batch_size*batch:batch_size*(batch+1)],
                                             y: y_train[batch_size*batch:batch_size*(batch+1)]}
                                 ),
                         sess.run(accuracy,
                                  feed_dict={x:x_train[batch_size*batch:batch_size*(batch+1)],
                                             y: y_train[batch_size*batch:batch_size*(batch+1)]}
                                 )
                        )
                 )
            sess.run(train_step,
                     feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], 
                                y: y_train[batch_size*batch:batch_size*(batch+1)]
                               }
                    )
        print("{} epochs completed and test accuracy is {}".
              format(sess.run(epoch_run)+i+1,sess.run(accuracy,feed_dict={x:x_test,
                                                      y:y_test})))
        sess.run(epoch_run.assign(epoch_run+epochs))
        
        save_path = saver.save(sess, "/home/aman/dogs/OpenCV/models/"+str(img_size)+"model.ckpt")
        print("Model saved in path: %s" % save_path)
writer.close()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Restoring parameters from /home/aman/dogs/OpenCV/models/50model.ckpt
Model restored.
epoch 12 and mini-batch 1 : Cross-entropy error is 4.70760393143 and training accuracy is 0.0533333346248
epoch 12 and mini-batch 2 : Cross-entropy error is 4.70036315918 and training accuracy is 0.0333333350718
epoch 12 and mini-batch 3 : Cross-entropy error is 4.49260807037 and training accuracy is 0.0466666668653
epoch 12 and mini-batch 4 : Cross-entropy error is 4.77154541016 and training accuracy is 0.0
epoch 12 and mini-batch 5 : Cross-entropy error is 4.68791770935 and training accuracy is 0.0333333350718
epoch 12 and mini-batch 6 : Cross-entropy error is 4.69832849503 and training accuracy is 0.0466666668653
epoch 12 and mini-batch 7 : Cross-entropy error is 4.68750715256 and training accur

epoch 12 and mini-batch 74 : Cross-entropy error is 4.67013263702 and training accuracy is 0.019999999553
epoch 12 and mini-batch 75 : Cross-entropy error is 4.77254295349 and training accuracy is 0.019999999553
epoch 12 and mini-batch 76 : Cross-entropy error is 4.73412370682 and training accuracy is 0.019999999553
epoch 12 and mini-batch 77 : Cross-entropy error is 4.80779981613 and training accuracy is 0.0333333350718
epoch 12 and mini-batch 78 : Cross-entropy error is 4.78271055222 and training accuracy is 0.0066666668281
epoch 12 and mini-batch 79 : Cross-entropy error is 4.68964099884 and training accuracy is 0.0333333350718
epoch 12 and mini-batch 80 : Cross-entropy error is 4.52323865891 and training accuracy is 0.0733333304524
epoch 12 and mini-batch 81 : Cross-entropy error is 4.68006038666 and training accuracy is 0.0266666673124
epoch 12 and mini-batch 82 : Cross-entropy error is 4.71024417877 and training accuracy is 0.0133333336562
epoch 12 and mini-batch 83 : Cross-entro

epoch 15 and mini-batch 30 : Cross-entropy error is 4.59755516052 and training accuracy is 0.0333333350718
epoch 15 and mini-batch 31 : Cross-entropy error is 4.69286346436 and training accuracy is 0.019999999553
epoch 15 and mini-batch 32 : Cross-entropy error is 4.66377544403 and training accuracy is 0.019999999553
epoch 15 and mini-batch 33 : Cross-entropy error is 4.65693378448 and training accuracy is 0.0466666668653
epoch 15 and mini-batch 34 : Cross-entropy error is 4.69479370117 and training accuracy is 0.0266666673124
epoch 15 and mini-batch 35 : Cross-entropy error is 4.73283720016 and training accuracy is 0.019999999553
epoch 15 and mini-batch 36 : Cross-entropy error is 4.75030422211 and training accuracy is 0.0333333350718
epoch 15 and mini-batch 37 : Cross-entropy error is 4.68344688416 and training accuracy is 0.019999999553
epoch 15 and mini-batch 38 : Cross-entropy error is 4.65484476089 and training accuracy is 0.0599999986589
epoch 15 and mini-batch 39 : Cross-entrop

epoch 15 and mini-batch 108 : Cross-entropy error is 4.73684597015 and training accuracy is 0.019999999553
epoch 15 and mini-batch 109 : Cross-entropy error is 4.71311855316 and training accuracy is 0.0
epoch 15 and mini-batch 110 : Cross-entropy error is 4.71281147003 and training accuracy is 0.0266666673124
epoch 15 and mini-batch 111 : Cross-entropy error is 4.72118091583 and training accuracy is 0.0133333336562
epoch 15 and mini-batch 112 : Cross-entropy error is 4.7976641655 and training accuracy is 0.0066666668281
epoch 15 and mini-batch 113 : Cross-entropy error is 4.74967765808 and training accuracy is 0.0066666668281
epoch 15 and mini-batch 114 : Cross-entropy error is 4.72365188599 and training accuracy is 0.019999999553
epoch 15 and mini-batch 115 : Cross-entropy error is 4.73470449448 and training accuracy is 0.0133333336562
epoch 15 and mini-batch 116 : Cross-entropy error is 4.67795753479 and training accuracy is 0.019999999553
epoch 15 and mini-batch 117 : Cross-entropy 